# Design of an A/B Test : Free Trial Screener
by Julien Delbove




## Experiment Design
### Metric Choice

In the experiment, I will use the following invariant metric:
* Number of cookies
* Number of clicks
* Click-through-probability  

As evaluation metrics, I choose:
* Gross conversion
* Net conversion


I made these decisions for the following reasons:
* Number of cookies (Number of unique users to visit the course overview page): Since the unit of diversion is the cookies, they must be evenly distributed between the control and the experiment groups. Therefore, I will use this metric as an invariant.
* Number of user-ids (Number of users who enroll in the free trial): This metric is not an invariant since the experiment is expected to provide warning to the user and reduce the number of enrollment from people who can't commit enough hours. This metric can't be used as evaluation metric because it won't give information on the purpose of the experiment: How many of those users will continue past the free trial.
* Number of clicks (Number of unique cookies to click the "Start free trial" button):Since the experiment happen after the click on the "Start free trial", this metric can be used as an invariant.
* Click-through-probility (Number of unique cookies to click the "Start free trial" button divided by the number of unique cookies to view the course overview page): This metric is based on 2 metrics that are invariants. Therfore it is an invariant.
* Gross conversion (Number of user-ids to complete checkout and enroll in the free trial divided by the number of unique cookies to click the "Start free trial" button): This metric is an evaluation metric because the ratio difference between the experiment and the control group will give the impact of the warming on the people who decide to enroll in the free trial.
* Retention (Number of user-ids to remain enroll past the 14-day boundary divided by the number of user-ids to complete checkout): This metric could also be considered as an evaluation metrics since it measures the impact of the experiment on the ratio of users remaining enrolled pas the 14-day boundary. However, I will not use this evaluation metric since it is equal to $ \frac{Net\ conversion}{Gross\ conversion}$.
* Net conversion (Number of user-ids to remain enroll past the 14-day boundary divided by the number of unique cookies to click the "Start free trial" button). I choose this metric as an evaluation metric since it give the impact of the experiment on the people who remain enroll past the 14-day boundary.


The results I expect for my chosen evaluation metrics are the following:
* Gross conversion: if the experiment works as expected, this metric should be lower for the experiment group than for the control group. Indeed, once the pop-up as been displayed to the visitor, I expect, he will enroll only if he evaluates that he could commit the requested amount of time. Therefore, less visitors should enroll in the experiment group.
* Net conversion: I expect the net conversion to remain the same between the experiment and the control group. It measures how many people remains enroll after the 14-day boundary over the number of unique cookies to click the "Start free trial". In the experiment group, enrolled people have been warmed, therefore there will be less enrollment but it means also that more of them should remain enrolled after the 14-day boundary. In the control group, people are not warned, therefore there will be more enrollment but it means that more of them will leave the course in-between. At the end, the final number of people remaining engaged over the number of click on the "Start free trial" should be the same between the experiment and the control group. The advantage for Udacity is that udacity coaches will be less solicitated by people that are in the free trial period.


### Measuring Standard Deviation

In [1]:
import math

# Analytical standard deviation evaluation

# Inputs
unique_cookies_per_day=5000
click_through_probability_start_free_trial=0.08
probability_of_enrolling_given_click=0.20625
probability_of_payment_given_click=0.1093125

# Number of click on "Start free trial"
N=unique_cookies_per_day*click_through_probability_start_free_trial

# Evaluation of standard deviation of binomial distribution B(p,N)
standard_dev_gross_conversion=math.sqrt(
    probability_of_enrolling_given_click*(1.-probability_of_enrolling_given_click)/N)

standard_dev_net_conversion=math.sqrt(
    probability_of_payment_given_click*(1.-probability_of_payment_given_click)/N)

print "Standard deviation of Gross conversion for a sample size of 5000 cookies: %.4f"%standard_dev_gross_conversion
print "Standard deviation of Net conversion for a sample size of 5000 cookies: %.4f"%standard_dev_net_conversion


Standard deviation of Gross conversion for a sample size of 5000 cookies: 0.0202
Standard deviation of Net conversion for a sample size of 5000 cookies: 0.0156


The standard deviation for my evaluation metrics for a sample size of 5000 cookies are:
* 0.0202 for the Gross conversion
* 0.0156 for the Net conversion  

The two evaluation metric are probability. Their distributions are likely to follow a Binomial distribution. Therefore I expect the analytical estimate to be comparable to the empirical variability.

### Sizing
#### Number of Samples vs. Power  


In [2]:
# Number of sample for one group for each metric:
N1=25835
N2=27413

# Number of sample for both groups:
Ns=2*max(25835,27413)

# Number of pageviews
N=Ns/0.08

print "The number of pageviews necessary for the experiment is %i"%N

The number of pageviews necessary for the experiment is 685325


Since there are only 2 evaluations metrics, I will not use the Bonferroni correction during my analysis phase.

In order to compute the number of click on the "Start free trial" button that will be needed, I used the calculator available at the address: [Calculator](http://www.evanmiller.org/ab-testing/sample-size.html). 
I get the following result:
* 25835 samples necessary for the Gross conversion to identify a 1% variation with $\alpha=0.05$ and $\beta=0.2$.
* 27413 samples necessary for the Net conversion to identify a 1% variation with $\alpha=0.05$ and $\beta=0.2$.

In order to be able to get a proper evaluation of the evolution of both metrics, I will consider the maximum value of the figures obtained with the calculator. This number of sample is for one group. Since there are 2 groups and the pageviews will be balanced between both group, the number of samples for both groups is: $max(27413,25835)*2$. As there is Click-through-probability on "Start free trial" equal to $0.08$, the number of pageviews necessary is equal to: $N=\frac{max(27413,25835)*2}{0.08}$. It gives 685325 pageviews.

#### Duration vs. Exposure


In [3]:
# fraction of traffic to be diverted for a one month experiment
f=685325./(40000*30)*100

print "The fraction of the traffic for a 30 days experiment is equal to %.2f%%"%f

# Number of days with a fraction of 0.5
Nd=685325./(40000*0.5)

print "The number of days for a fraction of divertion equal to 0.5 days is equal to %.2f"%Nd

The fraction of the traffic for a 30 days experiment is equal to 57.11%
The number of days for a fraction of divertion equal to 0.5 days is equal to 34.27


In order to get an idea of the fraction of traffic that I would divert for the experiment, I considered a reasonable amount of time (around one month). It means that the fraction of traffic $f$  that needs to be diverted verify the relation: $f*40000*30=685325$. It gives $f=\frac{685325}{40000*30}$ which is around $57\%$.  

As an answer to the question, I considered a fraction of 50% (half of the traffic). It gives a number of days equal to $N_d=\frac{685325}{40000*0.5}$ which is equal to $N_d=34.27$. As the number of days is a integer value, the experiment should be done on 35 days for a fraction of divertion equal to 0.5.  

That the experiment consist only to warn users of a necessary commitment for the training, there is no risk for Udacity and the fraction can be chosen depending on the duration expected.

## Experiment Analysis
### Sanity Checks


In [4]:
import pandas as pd

control=pd.read_csv("Final Project Results - Control.csv", sep=',')
exp=pd.read_csv("Final Project Results - Experiment.csv", sep=',')

print control.head(5)
print "================================================="
print exp.head(5)


Npageview_c=control["Pageviews"].sum()
Npageview_e=exp["Pageviews"].sum()

Nclicks_c=control["Clicks"].sum()
Nclicks_e=exp["Clicks"].sum()

# Sanity check for Number of cookies
p_theory1=0.5
p_observed1=float(Npageview_c)/(Npageview_c+Npageview_e)
m1=1.96

SD1=math.sqrt(p_theory1*(1.-p_theory1)/(Npageview_c+Npageview_e))
LB1=p_theory1-m1*SD1
UB1=p_theory1+m1*SD1
print "\n"
print "Lower Bound for Number of cookies is %.4f"%LB1
print "Upper Bound for Number of cookies is %.4f"%UB1
print "Observed probability is %.4f"%p_observed1

# Sanity check for Number of clicks
p_theory2=0.5
p_observed2=float(Nclicks_c)/(Nclicks_c+Nclicks_e)
m2=1.96

SD2=math.sqrt(p_theory2*(1.-p_theory2)/(Nclicks_c+Nclicks_e))
LB2=p_theory2-m2*SD2
UB2=p_theory2+m2*SD2
print "\n"
print "Lower Bound for Number of clicks is %.4f"%LB2
print "Upper Bound for Number of clicks is %.4f"%UB2
print "Observed probability is %.4f"%p_observed2

# Sanity check for click-through-probability
p_theory3=0.
p_observed3=float(Nclicks_c)/Npageview_c-float(Nclicks_e)/Npageview_e
m3=1.96

p_pooled=float((Nclicks_c+Nclicks_e))/(Npageview_c+Npageview_e)
SD3=math.sqrt(p_pooled*(1.-p_pooled)*(1./Npageview_c+1./Npageview_e))
LB3=p_theory3-m3*SD3
UB3=p_theory3+m3*SD3
print "\n"
print "Lower Bound for difference of click-through-probability is %.4f"%LB3
print "Upper Bound for difference of click-through-probability is %.4f"%UB3
print "Observed difference of click-through-probability is %.4f"%p_observed3



          Date  Pageviews  Clicks  Enrollments  Payments
0  Sat, Oct 11       7723     687        134.0      70.0
1  Sun, Oct 12       9102     779        147.0      70.0
2  Mon, Oct 13      10511     909        167.0      95.0
3  Tue, Oct 14       9871     836        156.0     105.0
4  Wed, Oct 15      10014     837        163.0      64.0
          Date  Pageviews  Clicks  Enrollments  Payments
0  Sat, Oct 11       7716     686        105.0      34.0
1  Sun, Oct 12       9288     785        116.0      91.0
2  Mon, Oct 13      10480     884        145.0      79.0
3  Tue, Oct 14       9867     827        138.0      92.0
4  Wed, Oct 15       9793     832        140.0      94.0


Lower Bound for Number of cookies is 0.4988
Upper Bound for Number of cookies is 0.5012
Observed probability is 0.5006


Lower Bound for Number of clicks is 0.4959
Upper Bound for Number of clicks is 0.5041
Observed probability is 0.5005


Lower Bound for difference of click-through-probability is -0.0013
Upper B

The sanity checks for the invariant metrics give:
* Number of cookies
    * Lower Bound: 0.4988
    * Upper Bound: 0.5012
    * Observed Probability: 0.5006
    * Since the Observed Probability is between the lower and the upper bound of the 95% confidence interval, the sanity check pass
* Number of clicks:
    * Lower Bound: 0.4959
    * Upper Bound: 0.5041
    * Observed Probability: 0.5005
    * For the same reason as for the Number of cookies, the sanity check pass for the Number of clicks
* Click-through-probability
    * Lower Bound: -0.0013
    * Upper Bound: 0.0013
    * Observed Probability: -0.0001
    * The observed difference of click-through probability is between the lower and upper bound of the 95% confidence interval, so the sanity check pass.

### Result Analysis
#### Effect Size Tests

In [5]:
# Remove lines with NaN for evaluation Metrics
control_sub=control.dropna()
exp_sub=exp.dropna()

Nclicks_c=control_sub["Clicks"].sum()
Nclicks_e=exp_sub["Clicks"].sum()

Nenrollments_c=control_sub["Enrollments"].sum()
Nenrollments_e=exp_sub["Enrollments"].sum()

Npayments_c=control_sub["Payments"].sum()
Npayments_e=exp_sub["Payments"].sum()

# Evaluation of 95% confidence interval for Gross conversion
p_observed1=float(Nenrollments_e)/Nclicks_e-float(Nenrollments_c)/Nclicks_c
m1=1.96

p_pooled1=float((Nenrollments_c+Nenrollments_e))/(Nclicks_c+Nclicks_e)
SD1=math.sqrt(p_pooled1*(1.-p_pooled1)*(1./Nclicks_c+1./Nclicks_e))
LB1=p_observed1-m1*SD1
UB1=p_observed1+m1*SD1
print "\n"
print "Lower Bound for difference of Gross Conversion difference is %.4f"%LB1
print "Upper Bound for difference of Gross Conversion difference is %.4f"%UB1

# Evaluation of 95% confidence interval for Net conversion
p_observed2=float(Npayments_e)/Nclicks_e-float(Npayments_c)/Nclicks_c
m2=1.96

p_pooled2=float((Npayments_c+Npayments_e))/(Nclicks_c+Nclicks_e)
SD2=math.sqrt(p_pooled2*(1.-p_pooled2)*(1./Nclicks_c+1./Nclicks_e))
LB2=p_observed2-m2*SD2
UB2=p_observed2+m2*SD2
print "\n"
print "Lower Bound for difference of Net Conversion difference is %.4f"%LB2
print "Upper Bound for difference of Net Conversion difference is %.4f"%UB2





Lower Bound for difference of Gross Conversion difference is -0.0291
Upper Bound for difference of Gross Conversion difference is -0.0120


Lower Bound for difference of Net Conversion difference is -0.0116
Upper Bound for difference of Net Conversion difference is 0.0019


The results for the evaluation metrics are:
* Gross Conversion 
    * The Confidence Interval is [-0.0291,-0.0120]
    * The difference in the Gross Conversion probability is statistically significant since zero is not in the confidence interval
    * The result is also practically significant since $d_{min}$ is not in the interval
* Net conversion
    * The Confidence Interval is [-0.0116,0.0019]
    * The difference in the Net Conversion probability is not statistically significant since zero is included in the confidence interval.
    * The result is not practically significant since $d_{min}$ is included in the interval

#### Sign Tests


In [6]:
import numpy as np

control_sub.is_copy = False
exp_sub.is_copy = False

# Computation of Gross conversion probability per day
control_sub["Gross_Conversion"]=control_sub["Enrollments"]/control_sub["Clicks"]
#control_sub["Gross_Conversion"]=control_sub[:,"Enrollments"]/control_sub[:,"Clicks"]

exp_sub["Gross_Conversion"]=exp_sub["Enrollments"]/exp_sub["Clicks"]

# Computation of Net conversion probability per day
control_sub["Net_Conversion"]=control_sub["Payments"]/control_sub["Clicks"]
exp_sub["Net_Conversion"]=exp_sub["Payments"]/exp_sub["Clicks"]

# Computation sign tables
sign_diff_Gross_Conversion=np.sign(exp_sub["Gross_Conversion"]-control_sub["Gross_Conversion"])
sign_diff_Net_Conversion=np.sign(exp_sub["Net_Conversion"]-control_sub["Net_Conversion"])

print "Number of success for Gross Conversion: %i"%((sign_diff_Gross_Conversion==-1.).sum())
print "Number of success for Net Conversion: %i"%((sign_diff_Net_Conversion==-1.).sum())
print "Number of trials: %i"%(len(sign_diff_Gross_Conversion))


Number of success for Gross Conversion: 19
Number of success for Net Conversion: 13
Number of trials: 23


The sign tests for the evaluation metrics give the following results:
* Gross Conversion 
    * 19 successes (number of days where the probability of Enrollment given clicks for experiment group is below the one of the control group)
    * 23 days are recorded
    * with the p-value calculator proposed at the address: [p-value Calculator](http://graphpad.com/quickcalcs/binomial1.cfm), it gives the value 0.0026
    * As this value is below the $\alpha$ value, the result of the sign test is statistically significant.
* Net Conversion
    * 13 successes
    * 23 trials
    * p-value is equal to 0.6776
    * the difference is not statistically significant


#### Summary

I didn't use the Bonferroni correction because only 2 metrics are used for the evaluation of the experiment. Therefore it is not likely that to see a significant difference just by chance.  
The effect size hypothesis tests and the sign tests are coherents and give the same conclusions.


### Recommendation

The effect of the experiment seems to have exactly the expected impact:
* less people enroll for the free trial: the Gross conversion is statistically and practically lower which means that the warming on the mandatory commitment has the impact expectted.
* the number of people remaining enrolled after the 14-days is not significantly impacted with a confidence interval of 95%: the Net conversion is not statistically different in the experiment and in the control group. However, with a reduced confidence interval (50% for example), the experiment would show a statistical significant difference on the Net conversion.

Because of the statistically significant difference on the Net conversion for the 50% confidence interval, even if the difference might not be practically significant, I would recommend to not apply the change to the udacity website. 


## Follow-Up Experiment

The results obtained in the experiment have a real impact on the people who enroll in the course. However, since there is a high risk of having a decrease of the Net conversion, it also seems that the warning is a bit to hard: it prevents people that evaluate less than 5h per week to enroll whereas depending on their background, less than 5h might enough for them. 

As a follow up experiment I would slightly modify the warning by making the number of hour required depending on the background of the people. In addition to the number of hours committed to learning, I would add an additional question like: How many years of experience have you got in a related field (0, 1-3, 3+). For 0 years of experience in a related field, the minimum number of hours could become 7h, for 1-3 years of experience, 5 years could remains the limit. For people, with 3+ years of experience in related field (software development, mathematics, engineering,...), the minimum number of hour could be 3h.

As for the original experience, my hypothesis is that the gross conversion will decrease since clear expectations are set depending on the background of the people (maybe a bit less than the original experiment). But since the warning is better adapted to the people background, I expect that the Net conversion will remain the same: the warning should not prevent anybody with an adequate number of hours committed to learning with regards to their experience to enroll.

I would use the same metrics and the same units of diversion since the experiment is the same expect for the screener.
